In [2]:
from OmiDos import LoadSingleSample, seed_everything, OmiDosModel, ConvertATACTorchDataset, ConvertRNATorchDataset, PrepscATAC, RunDenoising, PrepscRNA, ConvertRNATorchBatchDataset, ConvertATACTorchBatchDataset
import os
import pandas as pd
import numpy as np
import scanpy as sc
from sklearn.cluster import KMeans
import datetime
import time
import tracemalloc
import psutil
tracemalloc.start()
process = psutil.Process(os.getpid())
# folder_path = "./data/scRNA/"
output_path = "./output/"
# feature = "human"
original_dir = "/home/fanyi/1_paper/multi_omics/code/Palate_analysis/scScalpChromatin-main/results"
# 获取该文件夹下所有文件
batch_size = 256
imp = False
Denoising = False
pretrain = False
lr:float = 0.0001
seed_everything(1234)
device = "cuda:2"
file1 = "Palate"

In [ ]:
print(file1)

if not os.path.exists(os.path.join(output_path, file1)):
    os.makedirs(os.path.join(output_path, file1))
if not os.path.exists(os.path.join(output_path, file1)+'/model/'):
    os.makedirs(os.path.join(output_path, file1)+'/model/')
adata_x = LoadSingleSample(original_dir + '/scRNA_data/RNA.h5ad', trans=False)
adata_y = LoadSingleSample(original_dir + '/scATAC_preprocessing/baseline_preprocessing/unfiltered_output/Tile.h5ad', trans=False)

In [ ]:
# adata_y.obs.index = adata_y.obs.index.str.split('#').str[1]
# adata_x.obs.index = adata_x.obs.index.str.split('_').str[2]

print(adata_x.shape)
print(adata_y.shape)

In [ ]:
common_indices = adata_y.obs.index.intersection(adata_x.obs.index)

# Subsetting the DataFrames to keep only the common indices
adata_x_common = adata_x.obs.loc[common_indices]
adata_y_common = adata_y.obs.loc[common_indices]
common_indices

In [ ]:
len(common_indices)

In [ ]:
adata_x = adata_x[adata_x.obs_names.isin(common_indices)].copy()
adata_x


In [ ]:
adata_y = adata_y[adata_y.obs_names.isin(common_indices)].copy()
adata_y

In [ ]:
########################
start_time = time.time()  
########################

if Denoising:
    print("Run denoising part on the scRNA dataset.")
    adata = adata_x.copy()
    inx, top_nvars, imputation_data = RunDenoising(adata, TF=False, DenosingModule_Dir=os.path.join(output_path, file1)+'/model/', 
                                                    original_dim=adata.shape[0], im_dim=8, num_epochs = 100, 
                                                    filter_num =5000, batch_size = 256, device = device, 
                                                    imputation=imp, sample_time=1)
    # save inx
    np.save(os.path.join(output_path, file1, file1 + '_inx.npy'), inx)
    # save top_nvars
    np.save(os.path.join(output_path, file1, file1 + '_top_nvars.npy'), top_nvars)
    # save imputation_data
    np.save(os.path.join(output_path, file1, file1 + '_imputation_data.npy'), imputation_data)
    if inx is not None:
        db_0 = adata[:,inx].copy()
    else:
        db_0 = adata
    gene_selection = np.zeros(db_0.shape[1], dtype=bool)
    gene_selection[top_nvars] = True
    db_0.var['selected_genes'] = gene_selection
    db_1 = db_0[:, db_0.var['selected_genes']]
    print('Denoising dataset shape: {}'.format(db_1.shape))
    print("Finished denoising part.")
    # using imputation data
    if imp:
        # adata = PrepscRNA(adata, size_factors=True, logtrans_input=False)
        adata = imputation_data[:, db_0.var['selected_genes']]
        min_cells = int(adata.shape[0] * 0.03) # 3% of the total number of cells
        min_genes = int(adata.shape[1] * 0.03) # 3% of the total number of genes
        sc.pp.filter_genes(db_1, min_counts=min_cells) # 1
        sc.pp.filter_cells(db_1, min_counts=min_genes)
        # Check which indices to keep in adata
        indices_to_keep = adata.obs_names.isin(db_1.obs_names)
        # Filter adata to only include these indices
        adata = adata[indices_to_keep]
        adata.layers["raw"] = db_1.X.copy()
        adata.obs['size_factors'] = db_1.obs.n_counts / np.median(db_1.obs.n_counts)
        sc.pp.normalize_total(adata, target_sum=1e4)
        adata_x = adata
    else:
        print("Don't imputation.")
        min_cells = int(db_1.shape[0] * 0.03) # 3% of the total number of cells
        min_genes = int(db_1.shape[1] * 0.03) # 3% of the total number of genes
        adata_x = PrepscRNA(db_1, min_cells=min_cells, min_genes=min_genes, size_factors=True, logtrans_input=True)
else:
    print("Don't denoising.")
    min_cells = int(adata_x.shape[0] * 0.03) # 3% of the total number of cells
    min_genes = int(adata_x.shape[1] * 0.03) # 3% of the total number of genes
    adata_x = PrepscRNA(adata_x, min_cells=min_cells, min_genes=0, size_factors=True, logtrans_input=True)
adata_y = PrepscATAC(adata_y, min_genes=2000, min_cells=30) # min_cell filter gene, min_genes filter cell


In [ ]:
# save adata
# adata_x.write(os.path.join(output_path, file1, file1 + '_preprocessed_x.h5ad'))
# adata_y.write(os.path.join(output_path, file1, file1 + '_preprocessed_y.h5ad'))
if 'batch' not in adata_x.obs:
    adata_x.obs['batch'] = 'RNA'
adata_x.obs['batch'] = adata_x.obs['batch'].astype('category')
adata_x.obsm['onehot'] = np.stack(pd.Series([np.array([0, 1]) for _ in range(len(adata_x.obs.index))], index=adata_x.obs.index).values)
if 'batch' not in adata_y.obs:
    adata_y.obs['batch'] = 'ATAC'
adata_y.obs['batch'] = adata_y.obs['batch'].astype('category')
adata_y.obsm['onehot'] = np.stack(pd.Series([np.array([1, 0]) for _ in range(len(adata_y.obs.index))], index=adata_y.obs.index).values)
trainloader_x = ConvertRNATorchBatchDataset(adata_x, batch_size=batch_size)
trainloader_y, testloader_y = ConvertATACTorchBatchDataset(adata_y, batch_size=batch_size)
cell_type = "scRNA_scATAC"
alfa1 = 0.5
beda1 = 1.1
delta1 = 1.1
epsi1 = 1.4
sita1 = 0.0

alfa2 = 0.2
beda2 = 0.8
delta2 = 1.1
epsi2 = 1.0
sita2 = 0.7
latent = 32
encode_dim = [256, 64] #1024, 128
decode_x_dim = [64, 256] #128, 1024
decode_y_dim = []
# input_dim = adata.shape[1]
dims = [adata_x.shape[1], adata_y.shape[1], latent, encode_dim, decode_x_dim, decode_y_dim]
model = ModelmultiDEPF(dims=dims, n_centroids=10, device=device, 
                    binary=True, cell_type=cell_type)
model


In [ ]:
model.init_gmm_params(testloader_y,  device=device)
model.fit(trainloader_x, trainloader_y, mode='source',
        lr=lr, 
        max_iter= 200,
        save_path=os.path.join(output_path, file1)+'/model/',
        alfa = alfa1, beda=beda1, delta=delta1, epsi=epsi1, sita=sita1
        )
model.fit(trainloader_x, trainloader_y, mode='target',
        lr=lr, 
        max_iter= 200,
        save_path=os.path.join(output_path, file1)+'/model/',
        alfa = alfa2, beda=beda2, delta=delta2, epsi=epsi2, sita=sita2
        )
#######################################
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

In [ ]:
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
print(f"Peak memory usage: {peak / 10**6} MB")

memory_info = process.memory_info()
rss_memory = memory_info.rss  # Resident Set Size
vms_memory = memory_info.vms  # Virtual Memory Size
print(f"RSS memory usage: {rss_memory / 10**6} MB")
print(f"VMS memory usage: {vms_memory / 10**6} MB")
#######################################

adata_x.obsm['latent_share'] = model.EmbedingSharescRNA(adata_x.X.toarray(), batch_size=batch_size).cpu().numpy()
adata_x.obsm['latent_Single'] = model.EmbedingSinglescRNA(adata_x.X.toarray(), batch_size=batch_size).cpu().numpy()


In [ ]:
adata_x.__dict__['_raw'].__dict__['_var'] = adata_x.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
adata_x.write(os.path.join(output_path, file1, file1 + '_Embeding_x.h5ad'))

In [ ]:
adata_y.obsm['latent_share'] = model.EmbedingSharescATAC(testloader_y, device=device, out='z')
adata_y.obsm['latent_single'] = model.EmbedingSinglescATAC(testloader_y, device=device, out='z')
adata_y.write(os.path.join(output_path, file1, file1 + '_Embeding_y.h5ad'))

In [ ]:
adata_x = sc.read_h5ad(os.path.join(output_path, file1, file1 + '_Embeding_x.h5ad'))

In [ ]:
adata_y = sc.read_h5ad(os.path.join(output_path, file1, file1 + '_Embeding_y.h5ad'))

In [ ]:
adata = adata_x
sc.pp.neighbors(adata, n_neighbors=30, use_rep='latent_share')
sc.tl.leiden(adata)
sc.tl.umap(adata, min_dist=0.2)
color = [c for c in ['leiden'] if c in adata.obs]
sc.pl.umap(adata, color=color, show=True, wspace=0.4, ncols=4)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=30, use_rep='latent_Single')
sc.tl.leiden(adata)
sc.tl.umap(adata, min_dist=0.2)
color = [c for c in ['leiden'] if c in adata.obs]
sc.pl.umap(adata, color=color, show=True, wspace=0.4, ncols=4)

In [ ]:
adata = adata_y
sc.pp.neighbors(adata, n_neighbors=30, use_rep='latent_share')
sc.tl.leiden(adata)
sc.tl.umap(adata, min_dist=0.2)
color = [c for c in ['leiden'] if c in adata.obs]
sc.pl.umap(adata, color=color, show=True, wspace=0.4, ncols=4)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=30, use_rep='latent_single')
sc.tl.leiden(adata)
sc.tl.umap(adata, min_dist=0.2)
color = [c for c in ['leiden'] if c in adata.obs]
sc.pl.umap(adata, color=color, show=True, wspace=0.4, ncols=4)

############

In [ ]:
adata = adata_x
sc.pp.neighbors(adata, n_neighbors=30, use_rep='latent_Single')
sc.tl.leiden(adata)
sc.tl.umap(adata, min_dist=0.2)
color = [c for c in ['leiden'] if c in adata.obs]
sc.pl.umap(adata, color=color, save=file1+"_"+datetime.datetime.now().strftime("%Y%m%d_%H%M%S")+"_Single_scRNA.pdf", show=True, wspace=0.4, ncols=4)

In [ ]:
adata = adata_x
sc.pp.neighbors(adata, n_neighbors=30, use_rep='latent_share')
sc.tl.leiden(adata, resolution=0.5)
sc.tl.umap(adata, min_dist=0.2)
color = [c for c in ['cell_type_rna', 'kmeans', 'leiden'] if c in adata.obs]
sc.pl.umap(adata, color=color, save=file1+"_"+datetime.datetime.now().strftime("%Y%m%d_%H%M%S")+"__Share_scRNA.pdf", show=True, wspace=0.4, ncols=4)

In [ ]:
pd.Series(adata.obs.index.values).to_csv(os.path.join(output_path, file1, file1 + '_selectCellid.csv'), index=False)

In [ ]:
# save adata.obs['leiden'] to csv
adata.obs['leiden'].to_csv(os.path.join(output_path, file1, file1 + '_Share_scRNA_leiden_clusters.csv'))

In [ ]:
adata = adata_y
sc.pp.neighbors(adata, n_neighbors=30, use_rep='latent_single')
sc.tl.leiden(adata, resolution=1.1)
sc.tl.umap(adata, min_dist=0.2)
color = [c for c in ['leiden'] if c in adata.obs]
sc.pl.umap(adata, color=color, save=file1+"_"+datetime.datetime.now().strftime("%Y%m%d_%H%M%S")+"_Single_scATAC.pdf", show=True, wspace=0.4, ncols=4)

In [ ]:
adata = adata_y
sc.pp.neighbors(adata, n_neighbors=30, use_rep='latent_share')
sc.tl.leiden(adata, resolution=1.1)
sc.tl.umap(adata, min_dist=0.2)
color = [c for c in ['leiden'] if c in adata.obs]
sc.pl.umap(adata, color=color, save=file1+"_"+datetime.datetime.now().strftime("%Y%m%d_%H%M%S")+"_Share_scATAC.pdf", show=True, wspace=0.4, ncols=4)

In [ ]:
# save adata.obs['leiden'] to csv
adata.obs['leiden'].to_csv(os.path.join(output_path, file1, file1 + '_Share_scATAC_leiden_clusters.csv'))

In [ ]:
import pandas as pd

# 假设 adata_y 是你的AnnData对象
# 提取 latent_share 数据
latent_share_data = adata_y.obsm['latent_share']

# 为列名生成一个列表，假设有多个特征
column_names = [f"latent{i+1}" for i in range(latent_share_data.shape[1])]

# 将 numpy array 转换为 DataFrame
latent_share_df = pd.DataFrame(latent_share_data, columns=column_names)

# 设置DataFrame的行名为adata_y.obs中的某个列，假设这个列名是'cell_id'
latent_share_df.index = adata_y.obs.index

# 保存 DataFrame 到 CSV 文件
latent_share_df.to_csv(os.path.join(output_path, file1, file1 + "_latent_share_data_ATAC.csv"))

print("Data saved to 'latent_share_data.csv', with cell IDs and dynamically specified column names.")


In [ ]:
import pandas as pd

# 假设 adata_y 是你的AnnData对象
# 提取 latent_share 数据
latent_share_data = adata_y.obsm['latent_single']

# 为列名生成一个列表，假设有多个特征
column_names = [f"latent{i+1}" for i in range(latent_share_data.shape[1])]

# 将 numpy array 转换为 DataFrame
latent_share_df = pd.DataFrame(latent_share_data, columns=column_names)

# 设置DataFrame的行名为adata_y.obs中的某个列，假设这个列名是'cell_id'
latent_share_df.index = adata_y.obs.index

# 保存 DataFrame 到 CSV 文件
latent_share_df.to_csv(os.path.join(output_path, file1, file1 + "_latent_private_data_ATAC.csv"))

print("Data saved to 'latent_private_data.csv', with cell IDs and dynamically specified column names.")

In [ ]:
import pandas as pd

# 假设 adata_y 是你的AnnData对象
# 提取 latent_share 数据
latent_share_data = adata_x.obsm['latent_share']

# 为列名生成一个列表，假设有多个特征
column_names = [f"latent{i+1}" for i in range(latent_share_data.shape[1])]

# 将 numpy array 转换为 DataFrame
latent_share_df = pd.DataFrame(latent_share_data, columns=column_names)

# 设置DataFrame的行名为adata_y.obs中的某个列，假设这个列名是'cell_id'
latent_share_df.index = adata_x.obs.index

# 保存 DataFrame 到 CSV 文件
latent_share_df.to_csv(os.path.join(output_path, file1, file1 + "_latent_share_data_RNA.csv"))

print("Data saved to 'latent_share_data.csv', with cell IDs and dynamically specified column names.")